In [1]:
import pandas as pd
import numpy as np
import h5py
import matplotlib.pyplot as plt
import tensorflow as tf
import math
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


In [2]:
# Make Data-frame One-Hot encoded for given column names list
def one_hot_df(data_frame, one_hot_colnames=list()) :
    if len(one_hot_colnames) != 0:
        colnames = list(data_frame)
        hot_col = list()

        for hot in one_hot_colnames :
            if hot in colnames :
                hot_col.append(hot)
    else:
        hot_col = list(data_frame)
        
    if 'district' in hot_col :
        hot_col.remove('district')
    if 'state' in hot_col :
        hot_col.remove('state')
    if 'age' in hot_col:
        hot_col.remove('age')
            
    df = pd.get_dummies(data_frame, columns=hot_col)
    return (df)

In [4]:
data = pd.read_csv("22_AHS_COMB.csv", sep="|")
data.head()

/Users/himanshubabal/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,hh_id,client_hh_id,hl_id,state,district,rural,stratum_code,psu_id,house_no,house_hold_no,...,fidx,as,as_binned,rtelephoneno,isnewrecord,recordupdatedcount,recordstatus,schedule_id,year,id
0,28473.0,10129.0,NaN,22,3,1,1,157280582,633,1,...,NaN,NaN,NaN,NaN,NaN,1.0,2.0,1.0,3.0,41995957.0
1,28476.0,10130.0,NaN,22,3,1,1,157278213,634,1,...,NaN,NaN,NaN,NaN,NaN,1.0,2.0,1.0,3.0,41995958.0
2,28480.0,10131.0,NaN,22,3,1,1,157278379,634,1,...,NaN,NaN,NaN,NaN,NaN,1.0,2.0,1.0,3.0,41995959.0
3,28477.0,10132.0,NaN,22,3,1,1,157281242,634,1,...,NaN,NaN,NaN,NaN,NaN,1.0,2.0,1.0,3.0,41995960.0
4,28478.0,10133.0,NaN,22,3,1,1,157280945,634,1,...,NaN,NaN,NaN,NaN,NaN,1.0,2.0,1.0,3.0,41995961.0


In [5]:
data.shape

(4155187, 99)

In [6]:
AHS_struct_workbook = pd.ExcelFile("Data_structure_AHS.xlsx")
AHS_struct_sheets_names = AHS_struct_workbook.sheet_names

In [7]:
def get_sheet_field_names(excel_workbook, sheet_name) :
    # Start from row 3, as initial 2 rows contain no info
    sheet = excel_workbook.parse(sheet_name, skiprows=2, na_values=['NA'])
    # Find index of 'NOTES:' in 1st cloumn and delete all rows below it
    notes_index = sheet.loc[sheet['Field Order'] == "NOTES:"].index.tolist()[0]
    sheet = sheet.ix[1 : notes_index - 1]
    
    # select column 2,3 and 4 (Filed name, Description and Codes used)
    sheet = sheet[[1,2,3]]
    # Remove <NaN> from Field Names
    sheet = sheet.dropna(subset=[list(sheet)[0]])
    
    # Selecting Non-Yellow field names
    # Dropping <NaN> from Field Descriptions and Codes Used
    sheet_non_yellow = sheet.dropna(subset=[list(sheet)[1], list(sheet)[2]])
    
    # Selecting 'None' and Non-'None' Codes used
    sheet_code_not_none = sheet_non_yellow[sheet_non_yellow['Codes Used'] != "None"]
    sheet_code_none = sheet_non_yellow[sheet_non_yellow['Codes Used'] == "None"]
    
    # Convert all 'Field Names' to list()
    sheet_all = sheet['Field Name'].tolist()
    sheet_non_yellow = sheet_non_yellow['Field Name'].tolist()
    sheet_yellow = list(set(sheet_all) - set(sheet_non_yellow))
    sheet_code_not_none = sheet_code_not_none['Field Name'].tolist()
    sheet_code_none = sheet_code_none['Field Name'].tolist()
    
    # Output in form of list() of lists()
    output = list()
    output.append(sheet_yellow)
    output.append(sheet_non_yellow)
    output.append(sheet_all)
    output.append(sheet_code_none)
    output.append(sheet_code_not_none)
    # output = list[sheet_yellow, sheet_non_yellow, sheet_all, sheet_code_none, sheet_code_not_none]
    
    return(output)

def lowercase_32Char(list_):
    list_1 = [x.lower() for x in list_]
    list_2 = [x[0:32] for x in list_1]
    return (list_2)

def lowercase_32Char_list(field_list) :
    # Field names in CSV files are max upto 32 characters
    # and all small letters
    l = len(field_list)
    sol = list()
    
    for field in field_list:
        sol.append(lowercase_32Char(field))
    
    return(sol)

# Remove yellow fields from the data frame
def remove_yellow_fields(data_frame, yellow_field_list) :
    df_col_names = list(data_frame)
    
    drop_col = list()
    for yellow in yellow_field_list :
        if yellow in df_col_names :
            drop_col.append(yellow)
            
    df = data_frame.drop(drop_col, axis=1)
    
    if 'id' in list(df) :
        df = df.drop(['id'], axis=1)
    
    return df

def sort_dataset_state_dist_house(data_frame) :
    return (data_frame.sort(['state', 'district', 'house_no', 'house_hold_no'])).reset_index(drop=True)

In [22]:
comb_field_list = lowercase_32Char_list(get_sheet_field_names(AHS_struct_workbook, "COMB"))

data_clean = remove_yellow_fields(data, comb_field_list[0])
data_clean_sorted = sort_dataset_state_dist_house(data_clean)

/Users/himanshubabal/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:72: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [23]:
data_clean_sorted.shape

(4155187, 77)

In [24]:
data_clean_sorted.head()

,state,district,rural,stratum_code,psu_id,house_no,house_hold_no,sex,usual_residance,relation_to_head,...,land_possessed,residancial_status,iscoveredbyhealthscheme,healthscheme_1,healthscheme_2,housestatus,householdstatus,isheadchanged,as_binned,year
0,22,1,1,1,156930595,1,1,1,1.0,1.0,...,1.0,1.0,2.0,NaN,NaN,1.0,1.0,2.0,NaN,2.0
1,22,1,1,1,156930650,1,1,2,1.0,2.0,...,1.0,1.0,2.0,NaN,NaN,1.0,1.0,2.0,NaN,2.0
2,22,1,1,1,156932015,1,1,2,NaN,3.0,...,1.0,1.0,2.0,NaN,NaN,1.0,1.0,2.0,NaN,2.0
3,22,1,1,1,156932129,1,1,1,NaN,3.0,...,1.0,1.0,2.0,NaN,NaN,1.0,1.0,2.0,NaN,2.0
4,22,1,1,1,156932243,1,1,2,NaN,3.0,...,1.0,1.0,2.0,NaN,NaN,1.0,1.0,2.0,NaN,2.0


In [ ]:
data_clean_sorted.to_csv('22_AHS_COMB_Clean_Sorted.csv')